In [1]:
#Qmak_Calderdale_FDM_FULL_V1.0.ipynb
##last modified 20230309 

#Builds the Airedale SUS FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) # required for datetime

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Calderdale_FDM_Format"
targetdb <-'yhcr-prd-phm-bia-core.CB_FDM_SUS_Calderdale_V1'

targetdb <-gsub(' ','',targetdb)
print (targetdb)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] "yhcr-prd-phm-bia-core.CB_FDM_SUS_Calderdale_V1"


In [2]:
#build care sites just for Warehouse sites

#NOTE - THIS WHOLE SECTION NEEDS WORK

sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
#sql2<- paste('drop table if exists ' ,targetdb,'.tmp_orgs', sep = "")
#sql3<- paste('drop table if exists ' ,targetdb,'.tmp_orgs2', sep = "")

#make the temp org tables 
#sql4 <-paste('create table ' ,targetdb,'.tmp_orgs as select distinct IDOrganisation from ',sourcedb,'.tbl_SRCode', sep = "")
#sql5 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationDoneAt from ',sourcedb,'.tbl_SRCode', sep = "")
#print (sql5)
#sql6 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationRegisteredAt from ',sourcedb,'.tbl_SRCode', sep = "")
#sql7 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationVisibleTo from ',sourcedb,'.tbl_SRCode', sep = "")
#sql8 <-paste('create table ' ,targetdb,'.tmp_orgs2 as Select distinct IDOrganisation from ' ,targetdb,'.tmp_orgs', sep = "")
#sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CY_FDM_MASTER.care_site a ,',targetdb,'.tmp_orgs2 b where a.care_site_source_value = b.IDOrganisation and a.place_of_service_source_value = \'GP practice site\'', sep = "")
sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CB_FDM_MASTER.care_site a 
              where a.care_site_id = 23', sep = "")

#print (sql9)
#sql10 <-paste('delete from ' ,targetdb,'.care_site where care_site_id = 236 or care_site_id = 886' , sep = "")

#Now Run them 
tb1 <- bq_project_query(project_id, sql1)
#tb2 <- bq_project_query(project_id, sql2)
#tb3 <- bq_project_query(project_id, sql3)
#tb4 <- bq_project_query(project_id, sql4)
#tb5 <- bq_project_query(project_id, sql5)
#tb6 <- bq_project_query(project_id, sql6)
#tb7 <- bq_project_query(project_id, sql7)
#tb8 <- bq_project_query(project_id, sql8)
tb9 <- bq_project_query(project_id, sql9)
#tb10 <- bq_project_query(project_id, sql10)

eric <- paste("Done care site "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

[1] "Done care site 2024-01-24 14:35:58.950813"


In [5]:
#New section to build user table loop 

#Your loop references this table
#You run this to load the list of tables into the dataframe "table"

sql3 <-paste('select distinct table_name  from ' ,sourcedb,'.INFORMATION_SCHEMA.COLUMNS where table_name like \'tbl%\' and column_name = \'person_id\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table <- bq_table_download(tb3)
table 


eric <- paste("Done builder tables "  ,Sys.time() + hours(1) ,sep = "")
print(eric)


#This then loops through the data frame until the end
# uses the two variables table[[i, 1]] and table[[i, 2]]
# in the query

sql4 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb4 <-bq_project_query(project_id, sql4)


sql5 <-paste('create table ',targetdb,'.tmp_persons 
(person_id int64);', sep = "")
tb5 <-bq_project_query(project_id, sql5)
#print(tb5)


#Loops through all the data tables to build a list of unique person_ids from all of them 

for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_persons select distinct cast(person_id as int64)  from ' ,sourcedb,'.',table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[1, 1]]))
    #cat("\n")
}


eric <- paste("Done tmp persons "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#Now build the persons table 

sql11 <-paste('drop table if exists ' ,targetdb,'.person', sep = "")
tb11 <- bq_project_query(project_id, sql11)


sql12 <-paste('create table ' ,targetdb,'.person 
as 
SELECT distinct a.person_id
, gender_concept_id
, year_of_birth
, month_of_birth
, day_of_birth
, birth_datetime
, death_datetime
, race_concept_id
, ethnicity_concept_id
, location_id
, provider_id
, care_site_id
, person_source_value
, gender_source_value
, gender_source_concept_id
, race_source_value
, race_source_concept_id
, ethnicity_source_value
, ethnicity_source_concept_id 
FROM yhcr-prd-phm-bia-core.CB_FDM_MASTER.person a
,',targetdb,'.tmp_persons b where a.person_id =  b.person_id' , sep = "")

tb12 <- bq_project_query(project_id, sql12)
#print(sql12)

eric <- paste("Done persons "  ,Sys.time() + hours(1) ,sep = "")
print(eric)



#This query makes the observation period based on the persons table BUT adds in rules for this data extract.
#This query makes the observation period based on the persons table BUT adds in rules for this data extract.

#Qmak_PrimaryCareObservationPeriod_Part1

sql15 <-paste('drop table if exists ' ,targetdb,'.tmp_Eventdates' , sep = "")
sql16 <-paste('drop table if exists ' ,targetdb,'.tmp_EventdatesValid' , sep = "")
sql17 <-paste('truncate table ' ,targetdb,'.observation_period' , sep = "")

tb15 <-bq_project_query(project_id, sql15)
tb16 <-bq_project_query(project_id, sql16)
tb17 <-bq_project_query(project_id, sql17)

#This just builds a temp table for later use 
sql18 <-paste('create table ',targetdb,'.tmp_Eventdates 
(person_id int64
, EventDate Datetime) ' , sep = "")
tb18 <-bq_project_query(project_id, sql18)

eric <- paste("temp event dates built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#Loop through the user tables
#selecting person_id and date from and date to for each of them.

#start_date_first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_start_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("Start dates built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#Now do end date
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_end_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null ;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("End dates built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)


sql24 <-paste('create table ',targetdb,'.tmp_EventdatesValid
as
select distinct a.person_id
,cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a ,' 
,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is not null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= date_add(e.death_datetime, INTERVAL 42 day) 
and a.EventDate <= (Select max(cast(updated_date as date)) from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list 
where dataset = \'CB_STAGING_DATABASE_SUS_Calderdale_FDM_Format\')', sep = "")

tb24 <-bq_project_query(project_id, sql24)


#-- BUT this only includes persons with a death datetime
#-- So section below includes those with a null deathdatetime

sql25 <-paste('insert into ',targetdb,'.tmp_EventdatesValid
select distinct a.person_id
, cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a
,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= (Select  max(cast(updated_date as date)) from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list 
where dataset = \'CB_STAGING_DATABASE_SUS_Calderdale_FDM_Format\')', sep = "")

tb25 <-bq_project_query(project_id, sql25)

print("Temp valid dates built")

#third part pushes these into the observation_period table

sql26 <-paste('insert into ' ,targetdb,'.observation_period
select distinct
ROW_NUMBER() over (Order by person_id) as observation_period_id 
, person_id
, min(EventDate) as observation_period_start_date
, max(EventDate) as observation_period_end_date
,null as period_type_concept_id
from ',targetdb,'.tmp_EventdatesValid
group by person_id', sep = "")


tb26 <-bq_project_query(project_id, sql26)

print ("observation period built")

# Now remove from person table where there is no observation period
# This happens where there is patient data but no actual records 
# or there is patient data but no dob 

sql26a <-paste(' delete from ' ,targetdb,'.person where person_id
not in (select person_id from ', targetdb, ' . observation_period)', sep = "")

tb26a <-bq_project_query(project_id, sql26a) 

print ("invalid people removed")


eric <- paste("obs period built invalid people removed "  ,Sys.time() + hours(1) ,sep = "")
print(eric)


# As the Warehouse tables are all fdm ready we can just copy them into the FDM space
# once we've dropped them - if they exist ! 
# So drop loop next 

#start drop them first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('drop table if exists ',targetdb,'.', table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

eric <- paste("User tables removed "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#Now build them 
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('create table ',targetdb,'.',table[[i, 1]] ,' as select src.* from ' ,sourcedb,'.',table[[i, 1]],' src 
    , ',targetdb,'.observation_period obs where cast(src.person_id as int64)  = obs.person_id 
    and src.',table[[i, 1]],'_end_date <= obs.observation_period_end_date  
    and src.',table[[i, 1]],'_start_date >= obs.observation_period_start_date', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}


eric <- paste("User tables built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)

#But this only works for those tables with a tbl_prefix containing person_id 
# so we need to manually mop up the ones that don't.


table_name
<chr>
tbl_SUS_Calderdale_APC_CM_20170618_To_20220731
tbl_SUS_Calderdale_APC_CM_20220210_to_20231116_Part1
tbl_SUS_Calderdale_EC_Backward_Compatible_20170701_To_20220731
tbl_SUS_Calderdale_EC_Backward_Compatible_20220601_to_20230930
tbl_SUS_Calderdale_Outpatient_20180401_To_20220731
tbl_SUS_Calderdale_Outpatient_20220601_to_20230930


[1] "Done builder tables 2024-01-24 14:49:35.051717"
[1] "Done tmp persons 2024-01-24 14:50:10.360346"
[1] "Done persons 2024-01-24 14:50:15.617384"
[1] "temp event dates built 2024-01-24 14:50:18.612835"
[1] "Start dates built 2024-01-24 14:50:37.700255"
[1] "End dates built 2024-01-24 14:50:55.10168"
[1] "Temp valid dates built"
[1] "observation period built"
[1] "invalid people removed"
[1] "obs period built invalid people removed 2024-01-24 14:51:14.300728"
[1] "User tables removed 2024-01-24 14:51:18.702968"
[1] "User tables built 2024-01-24 14:52:20.677062"


In [3]:
print ("starting visit builders")

#VISITBuilders
#These are added value scripts that build pseudo visit occurences tables. 
#Due to the variances of the source data it may not be possible to build generic
#visit occurrence builders.

#This is a temporary table to bring together all visit information that we may be interested in

sql62 <-paste(' drop table if exists ' ,targetdb,'.tmp_visit_builder', sep = "")
tb62 <-bq_project_query(project_id, sql62) 

#Make the tmp_visit_builder_table

sql2 <-paste('create table if not exists ' ,targetdb,'.tmp_visit_builder
( source_table string,
  source_table_identifier string,
  person_id int64,
  visit_description string,
 visitstartdate datetime,
 visitenddate datetime,
 visitlocation string,
 visitstartplace string,
 visitendplace string);', sep = "")
tb2 <- bq_project_query(project_id, sql2)

eric <- paste("temp visit builder table built "  ,Sys.time() + hours(1) ,sep = "")
print(eric)


#So push into a tmp visit builder table the person_id, Event_date, 
## You need to so this for each source table manually now !

#1) tbl_SUS_Calderdale_APC_CM_20170618_To_20220731

sql63 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
select "tbl_SUS_Airedale_AE_20150401_to_20190131" as source_table,
pbr_spell_id as source_identifier,
src.person_id,
"A and E" as description,
tbl_SUS_Airedale_AE_20150401_to_20190131_start_date as start_date,
tbl_SUS_Airedale_AE_20150401_to_20190131_end_date as end_date,
"Airedale AE" as location,
ward_code_at_episode_start_date as start_place,
ward_code_at_episode_end_date as end_place
FROM ' ,targetdb,'.tbl_SUS_Airedale_AE_20150401_to_20190131 src
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where src.person_id = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_SUS_Calderdale_APC_CM_20170618_To_20220731_end_date <= obs.observation_period_end_date
  and src.tbl_SUS_Calderdale_APC_CM_20170618_To_20220731_start_date >= obs.observation_period_start_date', sep = "")

tb63 <- bq_project_query(project_id, sql63)
print("APC_CM built")

#THIS SECTION NEEDS COMPLETION !!

#Qapp_Calderdale_FDM_Visit_Occurrences_ALL


#amended to sql <-paste(' insert everything into the occurrences table 

sql84 <-paste(' truncate table ' ,targetdb,'.visit_occurrence ', sep = "")
tb84 <- bq_project_query(project_id, sql84)

sql85 <-paste(' insert into ' ,targetdb,'.visit_occurrence 
(      person_id
      ,visit_occurrence_id
      ,table
      ,clinicaldataid -- the actual rowidentifier from the source table 
      ,rowid
      ,visitstartdate
      ,visitstartdatetime
      ,visitenddate
      ,visitenddatetime
      ,visitstartplace
      ,visitendplace
      ,visitlocation
      ,care_site_id
      ,visitprovider
      ,provider_id
      ,visitdesc
      ,visit_source_concept_id
)
select distinct person_id
      ,FARM_FINGERPRINT(person_id || source_table_identifier)     as visit_occurrence_id
      ,source_table
      ,source_table_identifier  as clinicaldataid 
      ,source_table_identifier  as rowid
      ,cast(visitstartdate as date) as visitstartdate
      ,visitstartdate  as visitstartdatetime
      ,cast(visitenddate as date)  as visitenddate
      ,visitenddate as visitenddatetime
      ,visitstartplace as visitstartplace
      ,visitendplace  as visitendplace
      ,visitlocation  as visitlocation
      ,null  as care_site_id
      ,"null" as visitprovider
      ,null as provider_id
      ,visit_description as visitdesc
      ,38004693 as visit_source_concept_id
   from ' ,targetdb,'.tmp_visit_builder vb' , sep = "")

tb85 <- bq_project_query(project_id, sql85)

print ("visit occurrence  built")
#This section builds the imosphere version of the visit_occurrence table

In [5]:
sql90 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb90 <-bq_project_query(project_id, sql90)

sql91 <-paste('drop table if exists ',targetdb,'.tmp_Eventdates;', sep = "")
tb91 <-bq_project_query(project_id, sql91)

sql92 <-paste('drop table if exists ',targetdb,'.tmp_EventdatesValid;', sep = "")
tb92 <-bq_project_query(project_id, sql92)

print("temp tables dropped")

eric <- paste('FDM Build of',targetdb,'finished'  ,Sys.time() + hours(1) ,sep = "")
print(eric)

[1] "temp tables dropped"
[1] "FDM Build ofyhcr-prd-phm-bia-core.CB_FDM_SUS_Calderdale_V1finished2024-01-15 11:48:15.188044"


In [6]:
#section to update the CD FDM_Management details 

sql90 <-paste('update yhcr-prd-phm-bia-core.CB_FDM_Management.cb_fdm_details 
set fdm_build_date = "2024-01-24" 
where fdm_name = "CB_FDM_SUS_Calderdale_V1" ;', sep = "")
tb90 <-bq_project_query(project_id, sql90)


eric <- paste('Management updated '  ,Sys.time() + hours(1) ,sep = "")
print(eric)

[1] "Management updated 2024-01-24 15:19:02.692868"
